# Import block

In [37]:
from pymongo import MongoClient
from elasticsearch import Elasticsearch
from collections import defaultdict
from habanero import Crossref, counts, cn
import datetime

# The DOIBoost pipeline (for the golden standard)

In [28]:
class DOIBoostPipeline():
    def __init__(self, dataset = 'doiboost2018'):
        
        # get the mongo client
        client = MongoClient('mongo')

        # get the database
        metadatadb = client.get_database('narcis')

        # get the metadata collection
        self.metacollection = metadatadb[dataset]
        
        # get the elasticsearch connection
        es_host = "elasticsnarcis"
        es_local = Elasticsearch([es_host])
        self.es = es_local
        
        # set the elasticsearch index and doctype
        self.searchindex = 'grid'
        self.doctype = 'metadata'
    
    
    def metadata_retriever(self, DOI):
        """
        Retrieves metadata when an DOI is provided.

        @param  Collection   The metadata collection
        @param  string       The DOI
        @return dict         The corresponding metadata
        """
    
        # find the metadata with the DOI
        subset = self.metacollection.find({'doi': DOI}).limit(1)

        # return the first item
        for item in subset:
            return item

        # there was no hit for the given DOI
        return None
    
    def grid_ids_retriever(self, metadata):
        """
        Uses metadata to retrieve the GRID ids

        @param  dict    The metadata of a paper
        @return array   The list of GRID ids    
        """

        # initial list
        gridIDs = []

        # get the author information
        authors = metadata['authors']

        # loop over the authors in the list
        for author in authors:

            # get the affiliation(s) of the author
            affiliations = author['affiliations']

            # only continue if there is information about the affiliation
            if affiliations:

                # loop over the affiliation information
                for affiliation in affiliations:

                    # get the identifiers
                    identifiers = affiliation['identifiers']

                    # if one author doesn't have a GRID id, we can't be sure about the result
                    if len(identifiers) < 2:
                        return []

                    # get the value of the second item, which is always the GRID id
                    gridIDs.append(identifiers[1]['value'])

        # return the list of affiliations
        return gridIDs

    def classify(self, gridIDs):
        """
        Classify a research as Dutch/Non-Dutch based on the grid ids

        @param  array    The grid IDs of the authors
        @return boolean  True for Dutch, False for Non-Dutch
        """

        # we can only classify with 100% True Positives and True Negatives
        # if we have a complete list of GRID ids
        if not gridIDs:
            return None

        # Get GRID data
        for ID in gridIDs:
            res = self.es.search(index=self.searchindex, doc_type=self.doctype, 
                                 body={"query": {"match": {'ID': "%s" % ID }}})
            country = res['hits']['hits'][0]['_source']['Country']

            # Verify Dutch affiliation
            if country == 'Netherlands':
                return True

        return False
    
    def pipeline(self, DOIs):
        """
        The complete pipeline for identifying Dutch research using DOIBoost

        @param  Collection    The DOIBoost collection in MongoDB
        @param  array         The list of DOIs that need to be identified
        @return dict          A dictionary with {DOI: classification} pairs
        """

        # resulting dict
        result = {}

        # loop over the DOIs
        for DOI in DOIs:

            # retrieve the metadata
            metadata = self.metadata_retriever(DOI)

            # retrieve the GRID ids
            gridIDs = self.grid_ids_retriever(metadata)

            # skip if we don't have any GRID ids and can't be sure about the DOI identification
            if gridIDs:

                # classify the DOI based on the GRID ids
                classification = self.classify(gridIDs)

                # only add the classification if it's True or False
                if classification is None:
                    continue

                # add the classification to the resulting dict
                result[DOI] = classification

        # return the result
        return result

# The CrossRef pipeline

In [29]:
class CrossRefPipeline():
    def __init__(self):
        
        # get the CrossRef API
        self.cr = Crossref()
        
        # get the elasticsearch connection
        es_host = "elasticsnarcis"
        es_local = Elasticsearch([es_host])
        self.es = es_local
        
        # set the elasticsearch index and doctype
        self.searchindex = 'authors'
        self.doctype = 'metadata'

        
    def metadata_retriever(self, DOI):
        """
        Retrieves CrossRef metadata when an DOI is provided.

        @param  string       The DOI
        @return dict         The corresponding CrossRef metadata
        """

        # find the metadata with the DOI
        research = self.cr.works(ids = DOI)

        # return the metadata
        return research['message']
    
    def author_retriever(self, metadata):
        """
        Retrieves the author names from the metadata and convert it to a string

        @param  dict    The CrossRef metadata
        @return array   The list of authors
        """

        # get the authors
        authors = metadata.get('author', None)

        # stop if the metadata doesn't contain an author
        if not authors:
            return None

        # the resulting list
        result = []

        # loop over the authors
        for author in authors:

            # retrieve the first name
            name = author.get('given', None)

            # retrieve the surname
            surname = author.get('family', None)

            # don't add the name if the surname is absent
            if not surname:
                continue

            # add the first name before the surname if there is one
            if name:
                fullname = name + " " + surname
            else:
                fullname = surname

            # add the name to the list
            result.append(fullname)

        # return the list of author names
        return result
    
    def retrieve_countries(self, authors, metadata):
        """
        Get the country of the authors

        @param  array   A list of author names
        @param  dict    The CrossRef metadata that contains the date of the research
        @return array   The list of countries
        """

        # resulting list
        result = []

        # loop over the authors
        for author in authors:

            # try to find the name in the author dataset
            try:
                res = es.search(index=self.searchindex, doc_type=self.doctype, 
                                body={"query": {"match": {'name': "%s" % author}}})
                hits = res['hits']['hits']
            except:
                continue

            # there are multiple hits
            if len(hits) > 1:

                # loop over the hits
                for hit in hits:

                    # add the first hit and move on to the next author
                    result.append(hit['_source']['country'])
                    break

            # there is one hit, add it
            else: 
                result.append(hits['_source']['country'])

        # return the list of countries
        return result
    
    def classify(self, countries):
        """
        Classify the DOI as Dutch/Non Dutch

        @param  array     The countries of the affiliations the authors work for
        @return boolean   True for Dutch, False for Non Dutch
        """

        # one of the countries has to be 'Netherlands' for the DOI to be classified as Dutch
        if 'Netherlands' in countries:
            return True
        return False
    
    def pipeline(self, DOIs):
        """
        The complete pipeline for identifying Dutch research using DOIBoost

        @param  Collection    The DOIBoost collection in MongoDB
        @param  array         The list of DOIs that need to be identified
        @return dict          A dictionary with {DOI: classification} pairs
        """

        # resulting dict
        result = {}

        # loop over the DOIs
        for DOI in DOIs:

            # retrieve the metadata
            metadata = self.metadata_retriever(DOI)
            
            # retrieve the authors from the metadata
            authors = self.author_retriever(metadata)

            # we need to have authors to continue
            if authors:

                # retrieve the countries of the authors
                countries = self.retrieve_countries(authors, metadata)
                
                # classify the DOI based on the GRID ids
                classification = self.classify(countries)

                # add the classification to the resulting dict
                result[DOI] = classification

        # return the result
        return result

# The evaluation function

In [30]:
def evaluation(classifications, golden):
    """
    Evaluate the classifier
    
    @param  dict   The labels from the classifier
    @param  dict   The golden standard labels
    @return dict   The True Positives, False Positives, True Negatives and False Negatives
    """
    
    # the resulting dict
    result = defaultdict(int)
    
    # loop over the DOIs
    for DOI in classifications:
        
        # get the label
        classification = classifications[DOI]
        
        # get the golden label
        gold = golden.get(DOI, None)
        
        # unknown if there is no golden label for the DOI
        if (gold is None):
            result['Unknown'] += 1
        
        # the label should be Positive
        elif gold == True:
            
            # True Positive
            if classification == True:
                result['TP'] += 1
            
            # False Negative
            else: 
                result['FN'] += 1
        
        # the label should be Negative
        else:
            
            # True Negative
            if classification == False:
                result['TN'] += 1
            
            # False Positive
            else: 
                result['FP'] += 1
       
    # return the result
    return result

# Lets see the model in action

In [36]:
DOIs = ['10.1007/s00422-017-0730-1']

In [31]:
D = DOIBoostPipeline('doiboost2017')
doiboost_labels = D.pipeline(DOIs)

In [32]:
C = CrossRefPipeline()
crossref_labels = C.pipeline(DOIs)

In [35]:
evalutation_results = evaluation(crossref_labels, doiboost_labels)
print(evalutation_results)

defaultdict(<class 'int'>, {'TN': 1})
